# Otimização Usando Pyomo

Neste exemplo vamos ver em um exemplo bem simples as bases do Pyomo!

## A problemática

Esta problemática se resume em achar a solução para o seguinte problema de produção:
    
Uma empresa produz 2 produtos, que vamos chamar de X e Y. Para produzir X a empresa gasta 20 horas de produção e a
demanda deste produto é cerca de 40 unidades anuais, a unidade deste produto rende 1000 reais para a empresa.
Já para o produto Y, são necessárias 30 horas para produzi-lo e a demanda é de 30 unidades anuais, cada unidade
dando um lucro de 1800 reais para a empresa. A empresa como um todo dispõe apenas de 1200 horas de produção anual
para qualquer um dos produtos X ou Y.

Agora precisamos de um modelo de produção que nos ajude a ver quantas unidades de X e quantas unidades de Y a dita
empresa irá produzir dentro daquele ano para ter o maior lucro possível. E qual será esse lucro.

## A resolução

Esse é um proplema clássico de planejamento da produção usando otimização matemática. Para resolvermos usando Pyomo
vamos precisar primeiro modelar o problema matematicamente. Precisaremos encontrar as variáveis do problema, 
a função objetivo e as restrições do conjunto. Abaixo vamos importar o Pyomo e então fazer a modelagem do problema.

In [3]:
import pyomo.environ as pyEnv

Vamos agora iniciar o nosso modelo matemático no pyomo, aqui vamos usar o `ConcreteModel()`. 

In [6]:
model = pyEnv.ConcreteModel()

### Variáveis

O problema pode ser modelado da seguinte forma:

X -> Variável responsável pelas unidades produzidas de X

Y -> Variável responsável pelas unidades produzidas de Y

As regiões de possíveis soluções está nos Reais não negativos. Geralmente os problemas não admitem valores negativos para as variáveis, pois não faz sentido ter uma unidade negativa. Outros modelos já admitem esse comportamento, tudo só depende do que você está modelando.

In [8]:
model.x = pyEnv.Var(within=pyEnv.NonNegativeReals)
model.y = pyEnv.Var(within=pyEnv.NonNegativeReals)

### Restrições

Agora nós vamos modelar as restrições do problema. As restrições vamo limitar meu conjunto solução, sem essa limitação talvez não exista uma solução bem definida.

1) Restrição de tempo: Sabemos que a empresa não tem todo o tempo do mundo para produzir X e Y, lembrando que leva
20h para produzir X e 30 para produzir Y, o total de tempo de produção de ambos não pode ultrapassar 1200

In [9]:
model.const1 = pyEnv.Constraint(rule=lambda model: (20*model.x + 30*model.y <= 1200))

2) A demanda para os produtos X e Y limitam minha produção, por que se eu produzir mais do que a demanda então
esses produtos não serão vendidos, forçando a empresa a fazer estoque, o que para nosso problema não é bom

In [10]:
model.const2A = pyEnv.Constraint(rule=lambda model: (model.x <= 40))
model.const2B = pyEnv.Constraint(rule=lambda model: (model.y <= 30))

### Função Objetivo

A parte mais importante do problema é a função objetivo, a função que vamos tentar otimizar. Nosso objetivo é
maximizar o lucro, que é dado pela quantidade produzida de X e Y

In [13]:
model.obj = pyEnv.Objective(rule=lambda model: 1000*model.x + 1800*model.y, sense=pyEnv.maximize)

### Resolução

O modelo está pronto agora nós só precisamos resolve-los. Vamos fazer isso usando uma classe do Pyomo chamada
`SolverFactory()` que vai conectar nosso modelo ao resolvedor externo. Esse resolvedor pode ser o `GLPK` ou `CPLEX`

In [15]:
opt = pyEnv.SolverFactory("glpk")
results = opt.solve(model)

A solução está pronta! Agora vamos printar nossas respostas de uma maneira mais inteligente

In [19]:
model.display()

Model unknown

  Variables:
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  30.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 69000.0

  Constraints:
    const1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1200.0 : 1200.0
    const2A : Size=1
        Key  : Lower : Body : Upper
        None :  None : 15.0 :  40.0
    const2B : Size=1
        Key  : Lower : Body : Upper
        None :  None : 30.0 :  30.0


Nossos valores ótimos para o problema é 15 unidades de X, 30 unidades de Y, retornando um lucro de 69mil reais
para a empresa. Como podemos ver também todas as restrições foram atendidasme 100% da capacidade produtiva foi
utilizada! Então esse modelo está muito bom!

#### Igor Souza e Helano Pessoa - Otimização Matemática usando Pyomo